In [ ]:
def TEForecast(ticker, dates, values ):    

    url = 'https://api.tradingeconomics.com/historical/ticker/{tick}?client=guest:guest'.format(tick = ticker)
    myDF = pd.read_json(url)
    myDF['DateTime'] = pd.to_datetime(myDF['DateTime'])
    myDF = myDF.set_index('DateTime')
    startDate = datetime.datetime.today() - relativedelta(years = 5)
    myDF = myDF[startDate:]
    myDF = myDF[['HistoricalDataSymbol', 'Value']]
    myDF = myDF.pivot_table(values='Value',
                            index=myDF.index,
                            columns='HistoricalDataSymbol',
                            aggfunc='first')    
    
    data = np.array([values]).T
    dates = [datetime.datetime.strptime(date, "%Y-%m-%d").date() for date in dates]
    points = pd.DataFrame(data,
                          index = dates,
                          columns = [ticker])
    
    lastRowOriginal = myDF.tail(1)
    forc = [lastRowOriginal, points]
    forecasted = pd.concat(forc).sum(axis = 1)
    forecasted.index = pd.to_datetime(forecasted.index)
    forcBase = forecasted 

    forecasted = forecasted.reindex(pd.date_range(start=forecasted.index[0],
                                                  end=forecasted.index[-1],
                                                  freq='B'))  
    forecasted = forecasted.interpolate(method='spline', order = 3)
    noise = np.random.normal(scale = 0.05 * myDF.std(), size = len(forecasted))

    forcNoise = forecasted + noise
    forcNoise = forcNoise.to_frame()
    forcNoise = forcNoise.drop(forcBase.index)
    forcNoise = pd.concat([forcNoise, forcBase]).sort_index()
    print forcNoise
    results = {'historical':myDF, 'forecasted':forcNoise, 'points':points}
    
    return results